In [ ]:
import pickle
import pathSearch, db

coordinates = [
    [35.195, 136.85],  # 北西
    [35.13954, 136.92251],  # 南東
]
# 緯度・経度の最小値・最大値を抽出
y1, x1, y2, x2 = pathSearch.rectangleArea(coordinates)
link, length = db.getRectangleRoadData(y1, x1, y2, x2)
G = pathSearch.linkToGraph(link, length)
pathSearch.connectGraph(G)
with open('graph_15000_nagoya.pkl', 'wb') as f:
    pickle.dump(G, f)
print("G.nodes = " + str(G.number_of_nodes()))
print("G.edges = " + str(G.number_of_edges()))

fname="graph_5000_nagoya.pkl"
# 4km四方の範囲
coordinates = [
    [35.1860, 136.8700],  # 北西（左上）
    [35.1524, 136.9041]   # 南東（右下）
]
# G.nodes = 4578
# G.edges = 7257

fname="graph_10000_nagoya.pkl"
coordinates = [
    [35.19007, 136.85500],  # 北西（左上）
    [35.14473, 136.91344]   # 南東（右下）
]
# G.nodes = 10091
# G.edges = 16445

fname="graph_15000_nagoya.pkl"
coordinates = [
    [35.195, 136.85],  # 北西
    [35.13954, 136.92251],  # 南東
]
# G.nodes = 14438
# G.edges = 23568



In [2]:
import pathSearch
import time, random, pickle
import networkx as nx

def experiment(G, mode, points, p, R1, dicname):
    #グラフ生成
    with open(dicname, "rb") as f:
        len_dic = pickle.load(f)
    sp = pathSearch.ShortestPathFinder(G, len_dic=len_dic)
    nodes = [sp.nearestNode(p) for p in points]
    st = sp.nearestNode(coordinates[0])
    en = sp.nearestNode(coordinates[1])
    #R1計算
    moveDist = R1
    # moveDist = nx.shortest_path_length(G, st, en) * R1
    #クエリ生成(all destination = en)
    query = []
    for p in nodes:
        query.append((p, en))
    #ORIS heur
    if mode == 1:
        R = pathSearch.Routing(sp)
        start_time = time.time()
        path, len_, position, len_walk = R.find_optimal_stops(query, st, en, moveDist)
        exec = time.time() - start_time
        # print(f"ORIS heur\n{len_=}km, {len_walk=}km, {exec=}s")
        return len_, len_walk, exec
    #ORIS exact
    if mode == 2:
        Ropt = pathSearch.RoutingOptimal(sp)
        start_time = time.time()
        path, len_, position, len_walk = Ropt.find_optimal_stops(query, st, en, moveDist)
        exec = time.time() - start_time
        # print(f"ORIS exact\n{len_=}km, {len_walk=}km, {exec=}s")
        return len_, len_walk, exec
    #提案手法1(setcoverあり)
    if mode == 3:
        sp = pathSearch.ShortestPathFinder(G, len_dic=len_dic)
        start = time.time()
        path, len, positions_SRP, path_positions, len_walk = pathSearch.new_BusRouting(st, en, nodes, sp, moveDist)
        exec = time.time() - start
        # print(f"提案手法1(setcoverあり)\n{len=}km, {len_walk=}km, {exec=}s")
        return len, len_walk, exec
    #提案手法2(setcoverなし)
    if mode == 4:
        sp = pathSearch.ShortestPathFinder(G, len_dic=len_dic)
        start = time.time()
        path, len, positions_SRP, path_positions, len_walk = pathSearch.BusRouting(st, en, nodes, sp, moveDist)
        exec = time.time() - start
        # print(f"提案手法2(setcoverなし)\n{len=}km, {len_walk=}km, {exec=}s")
        return len, len_walk, exec

from tqdm.notebook import tqdm
if __name__ == "__main__":
    ###########################################################################

    fname="graph_5000_nagoya.pkl"
    # 4km四方の範囲
    coordinates = [
        [35.1860, 136.8700],  # 北西（左上）
        [35.1524, 136.9041]   # 南東（右下）
    ]

    ############################################################################
    # 緯度・経度の最小値・最大値を抽出
    lat_min = min(coordinates[0][0], coordinates[1][0])
    lat_max = max(coordinates[0][0], coordinates[1][0])
    lon_min = min(coordinates[0][1], coordinates[1][1])
    lon_max = max(coordinates[0][1], coordinates[1][1])

    print("グラフ読み込み開始")
    with open(fname, 'rb') as f:
        G = pickle.load(f)
    print("グラフ読み込み終了")
    print("G.nodes = " + str(G.number_of_nodes()))
    print("G.edges = " + str(G.number_of_edges()))

    p_default = 15
    p_test = [5,10,15,20,25,30]
    R1_default = 0.5
    R1_test = [0.2, 0.4, 0.6, 0.8, 1]
    mode_test = [1,3,4]

    ite = 5
    for p in tqdm(p_test, desc="p_testループ", unit="設定"):
        sum_len = [0]*5
        sum_walk = [0]*5
        sum_exec = [0]*5
        for i in tqdm(range(ite), desc=f"  クエリ数={p}", leave=False):
            # ランダム座標の生成
            points = [
                [random.uniform(lat_min, lat_max), random.uniform(lon_min, lon_max)]
                for _ in range(p)
            ]
            for mode in mode_test:
                len, len_walk, exec = experiment(G, mode, points, p, R1_default, fname.replace("graph_", "len_", 1))
                sum_len[mode] += len
                sum_walk[mode] += len_walk
                sum_exec[mode] += exec
        for mode in mode_test:
            ave_len = sum_len[mode] / ite
            ave_walk = sum_walk[mode] / ite
            ave_exec = sum_exec[mode] / ite
            print("---------------------")
            print("mode："+str(mode))
            print("クエリ数："+str(p))
            print("経路長："+str(round(ave_len,2))+"km")
            print("歩行距離："+str(round(ave_walk,2))+"km")
            print("実行時間："+str(round(ave_exec,2))+"s")
            


グラフ読み込み開始
グラフ読み込み終了
G.nodes = 4578
G.edges = 7257


p_testループ:   0%|          | 0/6 [00:00<?, ?設定/s]

  クエリ数=5:   0%|          | 0/5 [00:00<?, ?it/s]

---------------------
mode：1
クエリ数：5
経路長：5.44km
歩行距離：4.76km
実行時間：0.22s
---------------------
mode：3
クエリ数：5
経路長：7.85km
歩行距離：1.89km
実行時間：0.26s
---------------------
mode：4
クエリ数：5
経路長：7.38km
歩行距離：1.67km
実行時間：0.14s


  クエリ数=10:   0%|          | 0/5 [00:00<?, ?it/s]

ValueError: '[35.1858504, 136.8698461]' is not in list

In [1]:
import pickle, pathSearch
from collections import defaultdict
import networkx as nx
from tqdm.notebook import tqdm  # ★Jupyter用tqdm

fname = "graph_15000_nagoya.pkl"

with open(fname, 'rb') as f:
    G = pickle.load(f)

print("G.nodes = " + str(G.number_of_nodes()))
print("G.edges = " + str(G.number_of_edges()))

sp = pathSearch.ShortestPathFinder(G)
len_dic = defaultdict(dict)

# all_pairs_shortest_path_length のイテレータをそのまま tqdm に渡す
apsp = nx.all_pairs_dijkstra_path_length(G, weight="weight")

# total にノード数を指定すると、何ノード分終わったかが分かる
for u, dists in tqdm(apsp, total=G.number_of_nodes(), desc="APSP"):
    for v, dist in dists.items():
        # 無向グラフなのでキーを正規化（len_SP と同じルール）
        key1 = min(str(u), str(v))
        key2 = max(str(u), str(v))
        len_dic[key1][key2] = dist

dicname = fname.replace("graph_", "len_", 1)
with open(dicname, "wb") as f:
    pickle.dump(len_dic, f)

G.nodes = 14438
G.edges = 23568


APSP:   0%|          | 0/14438 [00:00<?, ?it/s]

In [6]:
import pickle
import pathSearch
with open("len_dic2.pkl", "rb") as f:
    len_dic = pickle.load(f)
with open('graph2.pkl', 'rb') as f:
    G = pickle.load(f)
sp = pathSearch.ShortestPathFinder(G, len_dic=len_dic)